<a href="https://colab.research.google.com/github/olinyoder2534/AIWritingDetector/blob/main/CreatingEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import spacy

In [ ]:
train = pd.read_csv('/content/trainClean.csv')
test = pd.read_csv('/content/testClean.csv')

In [ ]:
train = train.sample(frac=1).reset_index(drop=True)
test = test.sample(frac=1).reset_index(drop=True)

In [ ]:
train.head()

In [ ]:
test.head()

## Spacy Embeddings

In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
nlp = spacy.load("en_core_web_lg")

In [ ]:
train['SpacyVector'] = train['textNoPII'].apply(lambda text: nlp(text).vector)

In [ ]:
test['SpacyVector'] = test['text'].apply(lambda text: nlp(text).vector)

## Bert Embeddings

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

bert_preprocess = hub.KerasLayer(preprocess_url, name='bert_preprocess')
bert_encoder = hub.KerasLayer(encoder_url, name='bert_encoder')

In [ ]:
def get_embeddings(texts):
    text_input = tf.convert_to_tensor(texts, dtype=tf.string)
    preprocessed_text = bert_preprocess(text_input)
    outputs = bert_encoder(preprocessed_text)
    return outputs['pooled_output'].numpy()

In [ ]:
train['BertVector'] = train['textNoPII'].apply(lambda x: get_embeddings([x])[0])

In [ ]:
test['BertVector'] = test['text'].apply(lambda x: get_embeddings([x])[0])

In [ ]:
train['BertVector'] = train['BertVector'].apply(lambda x: np.fromstring(x[1:-1], dtype=float, sep=' '))
test['BertVector'] = test['BertVector'].apply(lambda x: np.fromstring(x[1:-1], dtype=float, sep=' '))

## Downloading New Data

In [ ]:
train.to_csv('trainCleanEmbeddings.csv', index = False)
test.to_csv('testCleanEmbeddings.csv', index = False)

In [ ]:
from google.colab import files
files.download('trainCleanEmbeddings.csv')
files.download('testCleanEmbeddings.csv')